## 4. Programmatically Accessing Geospatial Data Using APIs

After completing this tutorial, you will be able to:

- Extract geospatial (x,y) coordinate information embedded within a JSON hierarchical data structure.
- Convert data imported in JSON format into a Geopandas DataFrame.

In this lesson, you work with JSON data accessed via the Colorado information warehouse. The data will contain geospatial information nested within it that will allow us to create a map of the data.

## Working with Geospatial Data
Check out the map Colorado DWR Current Surface Water Conditions map.

https://data.colorado.gov/Water/DWR-Current-Surface-Water-Conditions-Map-Statewide/j5pc-4t32

Remember from the previous lesson, APIs can be used for many different things. Web developers (people who program and create web sites and cool applications) can use APIs to create user friendly interfaces - like the map in the previous example that allows us to look at and interact with data. These APIs are similar to, if not the same as, the ones that you often use to access data in Python.

n this lesson, you will access the data used to create the map at the link above using Python.

- The data that you will use are located here: [View JSON format data used to create surface water map.](https://data.colorado.gov/resource/j5pc-4t32.json)

- And you can learn more about the data here: [View CO Current water surface .](https://data.colorado.gov/Water/Current-Surface-Water-Conditions-in-Colorado/4yw9-a5y6)

In [1]:
import requests
import folium
import urllib
from pandas.io.json import json_normalize
import pandas as pd
import folium
from geopandas import GeoDataFrame
from shapely.geometry import Point

In [4]:
# Get URL
water_base_url = "https://data.colorado.gov/resource/j5pc-4t32.json?"
water_full_url = water_base_url + "station_status=Active" + "&county=BOULDER"
water_full_url

'https://data.colorado.gov/resource/j5pc-4t32.json?station_status=Active&county=BOULDER'

ATTENTION WINDOWS USERS: We have noticed a bug where on windows machines, sometimes the https URL doesn’t work. Instead try the same url as above but without the **s** - like this: water_base_url = "http://data.colorado.gov/resource/j5pc-4t32.json?" This change has resolved many issues on windows machines so give it a try if you are having problems with the API.

In [6]:
data = requests.get(water_full_url)
type(data.json())

list

In [7]:
data

<Response [200]>

Remember that the **JSON** structure supports hierarchical data and can be NESTED. If you look at the structure of the .json file below, you can see that the location object, is nested with three sub objects:

- latitude
- longitude
- needs_recoding

Since **data.json()** is a **list** you can print out just the first few items of the list to look at your data as a sanity check.

In [9]:
data.json()[:2] # dos primers

[{'station_name': 'PECK-PELLA AUGMENTATION RETURN',
  'div': '1',
  'location': {'latitude': '40.160705',
   'needs_recoding': False,
   'longitude': '-105.168471'},
  'dwr_abbrev': 'PECRTNCO',
  'data_source': 'Co. Division of Water Resources',
  'amount': '0.00',
  'station_type': 'Diversion',
  'wd': '5',
  'http_linkage': {'url': 'https://dwr.state.co.us/Tools/Stations/PECRTNCO'},
  'date_time': '2021-08-18T01:00:00.000',
  'county': 'BOULDER',
  'variable': 'DISCHRG',
  'station_status': 'Active'},
 {'station_name': 'HIGHLAND DITCH AT LYONS, CO',
  'div': '1',
  'location': {'latitude': '40.215043',
   'needs_recoding': False,
   'longitude': '-105.256017'},
  'dwr_abbrev': 'HIGHLDCO',
  'data_source': 'Co. Division of Water Resources',
  'amount': '86.30',
  'station_type': 'Diversion',
  'wd': '5',
  'http_linkage': {'url': 'https://dwr.state.co.us/Tools/Stations/HIGHLDCO'},
  'date_time': '2021-08-18T02:30:00.000',
  'county': 'BOULDER',
  'variable': 'DISCHRG',
  'stage': '1.2

## Convert JSON to Pandas DataFrame
Now that you have pulled down the data from the website, you have it in the **JSON** format. For the next step, you will use the **json_normalize()** function from the **Pandas** library to convert this data into a **Pandas DataFrame.**

This function helps organize and flatten data into a semi-structed table. To learn more, check out the documentation!



In [10]:
from pandas.io.json import json_normalize

In [11]:
result = json_normalize(data.json())

C:\Users\34639\AppData\Local\Temp/ipykernel_6428/3462806043.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  result = json_normalize(data.json())


In [12]:
result.head()

,station_name,div,dwr_abbrev,data_source,amount,station_type,wd,date_time,county,variable,station_status,location.latitude,location.needs_recoding,location.longitude,http_linkage.url,stage,usgs_station_id
0,PECK-PELLA AUGMENTATION RETURN,1,PECRTNCO,Co. Division of Water Resources,0.00,Diversion,5,2021-08-18T01:00:00.000,BOULDER,DISCHRG,Active,40.160705,False,-105.168471,https://dwr.state.co.us/Tools/Stations/PECRTNCO,NaN,NaN
1,"HIGHLAND DITCH AT LYONS, CO",1,HIGHLDCO,Co. Division of Water Resources,86.30,Diversion,5,2021-08-18T02:30:00.000,BOULDER,DISCHRG,Active,40.215043,False,-105.256017,https://dwr.state.co.us/Tools/Stations/HIGHLDCO,1.22,NaN
2,BOULDER-LARIMER BYPASS NEAR BERTHOUD,1,BOUBYPCO,Co. Division of Water Resources,4.38,Diversion,4,2021-08-18T02:15:00.000,BOULDER,DISCHRG,Active,40.258726,False,-105.175817,https://dwr.state.co.us/Tools/Stations/BOUBYPCO,0.41,NaN
3,"BOULDER CREEK NEAR ORODELL, CO.",1,BOCOROCO,Co. Division of Water Resources,36.10,Stream,6,2021-08-18T02:15:00.000,BOULDER,DISCHRG,Active,40.006534,False,-105.330525,https://dwr.state.co.us/Tools/Stations/BOCOROCO,2.00,06727000
4,PALMERTON DITCH,1,PALDITCO,"Cooperative Program of CDWR, NCWCD & SVLHWCD",16.06,Diversion,5,2021-08-18T02:00:00.000,BOULDER,DISCHRG,Active,40.212511,False,-105.251836,https://dwr.state.co.us/Tools/Stations/PALDITCO,0.65,NaN


In [13]:
type(result)

pandas.core.frame.DataFrame

In [14]:
result.columns

Index(['station_name', 'div', 'dwr_abbrev', 'data_source', 'amount',
       'station_type', 'wd', 'date_time', 'county', 'variable',
       'station_status', 'location.latitude', 'location.needs_recoding',
       'location.longitude', 'http_linkage.url', 'stage', 'usgs_station_id'],
      dtype='object')

## Data Cleaning for Visualization
Now you can clean up the data. **Notice that your longitude and latitude values are stored as strings**. Do you think can create a map if these values are stored as strings?

In [15]:
result['location.latitude'][0]

'40.160705'

You can convert the strings to type float as follows.

In [17]:
result['location.latitude'] = result['location.latitude'].astype(float)
result['location.longitude'] = result['location.longitude'].astype(float)

In [18]:
result['location.longitude'][0]

-105.168471

Now that you have numeric values for mapping, make sure that are are no missing values.

In [19]:
result.shape

(83, 17)

In [20]:
result['location.longitude'].isna().any()

False

In [21]:
result['location.latitude'].isna().any()

False

There are no nan values in this data. However, if there were, you could remove rows where a column has a nan value in a specific column with the following: result_nonan = result.dropna(subset=['location.longitude', 'location.latitude'])

## Df to Gdf and Data Visualization
You will use the **folium** package to visualize the data. **One approach you could take would be to convert your Pandas DataFrame to a Geopandas DataFrame for easy mapping.**

In [23]:
geometry = [Point(xy) for xy in zip(result['location.longitude'], result['location.latitude'])] # create col geometry
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(result, crs=crs, geometry=geometry) # df to gdf

C:\Users\34639\anaconda3\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [24]:
gdf.head()

,station_name,div,dwr_abbrev,data_source,amount,station_type,wd,date_time,county,variable,station_status,location.latitude,location.needs_recoding,location.longitude,http_linkage.url,stage,usgs_station_id,geometry
0,PECK-PELLA AUGMENTATION RETURN,1,PECRTNCO,Co. Division of Water Resources,0.00,Diversion,5,2021-08-18T01:00:00.000,BOULDER,DISCHRG,Active,40.160705,False,-105.168471,https://dwr.state.co.us/Tools/Stations/PECRTNCO,NaN,NaN,POINT (-105.16847 40.16071)
1,"HIGHLAND DITCH AT LYONS, CO",1,HIGHLDCO,Co. Division of Water Resources,86.30,Diversion,5,2021-08-18T02:30:00.000,BOULDER,DISCHRG,Active,40.215043,False,-105.256017,https://dwr.state.co.us/Tools/Stations/HIGHLDCO,1.22,NaN,POINT (-105.25602 40.21504)
2,BOULDER-LARIMER BYPASS NEAR BERTHOUD,1,BOUBYPCO,Co. Division of Water Resources,4.38,Diversion,4,2021-08-18T02:15:00.000,BOULDER,DISCHRG,Active,40.258726,False,-105.175817,https://dwr.state.co.us/Tools/Stations/BOUBYPCO,0.41,NaN,POINT (-105.17582 40.25873)
3,"BOULDER CREEK NEAR ORODELL, CO.",1,BOCOROCO,Co. Division of Water Resources,36.10,Stream,6,2021-08-18T02:15:00.000,BOULDER,DISCHRG,Active,40.006534,False,-105.330525,https://dwr.state.co.us/Tools/Stations/BOCOROCO,2.00,06727000,POINT (-105.33052 40.00653)
4,PALMERTON DITCH,1,PALDITCO,"Cooperative Program of CDWR, NCWCD & SVLHWCD",16.06,Diversion,5,2021-08-18T02:00:00.000,BOULDER,DISCHRG,Active,40.212511,False,-105.251836,https://dwr.state.co.us/Tools/Stations/PALDITCO,0.65,NaN,POINT (-105.25184 40.21251)


Then, you can **plot the data using the folium functions GeoJson() and add_to() to add the data from the Geopandas DataFrame to the map object.**

In [31]:
m = folium.Map([40.01, -105.27], zoom_start= 10, tiles='cartodbpositron')
folium.GeoJson(gdf).add_to(m)

m

You can also cluster the markers, and add a popup to each marker, so you can give your viewers more information about station: such as its name and the amount of precipitation measured.

For this example below, you will work with the Pandas DataFrame you originally created from the JSON, instead of the Geopandas GeoDataFrame.

In [28]:
# Get the latitude and longitude from result as a list
locations = result[['location.latitude', 'location.longitude']]
coords = locations.values.tolist()


In [30]:
from folium.plugins import MarkerCluster

m = folium.Map([40.01, -105.27], zoom_start= 10, tiles='cartodbpositron')

marker_cluster = MarkerCluster().add_to(m)

for point in range(0, len(coords)):
    folium.Marker(location = coords[point], 
                  popup= 'Name: ' + result['station_name'][point] + ' ' + 'Precip: ' + str(result['amount'][point])
                 ).add_to(marker_cluster)

m